In [1]:
CHROMA_PATH = "msd_chroma"
DATA_PATH = "data\msd"
CHUNK_SIZE = 10000
CHUNK_OVERLAP = 1000

In [2]:
from langchain_community.embeddings import HuggingFaceEmbeddings


In [3]:
def get_embeddings():
   model_kwargs = {'device': 'cuda'}
   embeddings_hf = HuggingFaceEmbeddings(
       model_name='intfloat/multilingual-e5-large',
       model_kwargs=model_kwargs
   )
   return embeddings_hf

In [4]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

from langchain_community.document_loaders import DedocPDFLoader

from langchain_community.document_loaders import PyPDFLoader

import glob 
files = glob.glob(DATA_PATH + r'\*.pdf', recursive=True)


In [5]:
def load_documents():
   loaders = [PyPDFLoader(fp) for fp in files]
   all_documents = []
   count = 0
   for loader in loaders:
      count += 1
      print(f'{count}/{len(loaders)}', "Loading raw document..." + loader.file_path)
      raw_documents = loader.load()

      print("Splitting text...")
      text_splitter = RecursiveCharacterTextSplitter(
         chunk_size=CHUNK_SIZE,
         chunk_overlap=CHUNK_OVERLAP,
         length_function=len,
         add_start_index=True,
      )
      documents = text_splitter.split_documents(raw_documents)
      all_documents.extend(documents)

  
   return all_documents

In [6]:
from langchain.docstore.document import Document

def get_fixed_documents(documents):
    fixed_documents = []
    count = 0 
    for document in documents:
        count += 1
        print(f'{count}/{len(documents)}', "Обработка чанка")
        metadata= document.metadata
        metadata.pop('rotated_page_angles', None)
        doc =  Document(page_content=document.page_content, metadata=metadata)
        fixed_documents.append(doc)
    return fixed_documents

In [7]:
import os
import shutil

def save_to_chroma(chunks: list):
    # Clear out the database first.
    if os.path.exists(CHROMA_PATH):
        shutil.rmtree(CHROMA_PATH)

    # Create a new DB from the documents.
    db = Chroma.from_documents(
        chunks, get_embeddings(), persist_directory=CHROMA_PATH
    )
    db.persist()
    print(f"Saved {len(chunks)} chunks to {CHROMA_PATH}.")

In [8]:
chunks = load_documents()

1/2775 Loading raw document...data\msd\Baylisascariasis.pdf
Splitting text...
2/2775 Loading raw document...data\msd\BRUE (быстро разрешившееся необъяснимое событие).pdf
Splitting text...
3/2775 Loading raw document...data\msd\Campylobacter и подобные инфекции.pdf
Splitting text...
4/2775 Loading raw document...data\msd\Chlamydia.pdf
Splitting text...
5/2775 Loading raw document...data\msd\Clostridioides (ранее, Clostridium) difficile - индуцированная диарея.pdf
Splitting text...
6/2775 Loading raw document...data\msd\Clostridium perfringens Пищевое отравление.pdf
Splitting text...
7/2775 Loading raw document...data\msd\COVID-19 во время беременности.pdf
Splitting text...
8/2775 Loading raw document...data\msd\COVID-19 и острое повреждение почек (ОПП).pdf
Splitting text...
9/2775 Loading raw document...data\msd\Cиндром Пейтца-Йегерса.pdf
Splitting text...
10/2775 Loading raw document...data\msd\Cиндром поликистозных яичников (СПЯ).pdf
Splitting text...
11/2775 Loading raw document...da

In [9]:
fixed_chunks=get_fixed_documents(chunks)

1/4286 Обработка чанка
2/4286 Обработка чанка
3/4286 Обработка чанка
4/4286 Обработка чанка
5/4286 Обработка чанка
6/4286 Обработка чанка
7/4286 Обработка чанка
8/4286 Обработка чанка
9/4286 Обработка чанка
10/4286 Обработка чанка
11/4286 Обработка чанка
12/4286 Обработка чанка
13/4286 Обработка чанка
14/4286 Обработка чанка
15/4286 Обработка чанка
16/4286 Обработка чанка
17/4286 Обработка чанка
18/4286 Обработка чанка
19/4286 Обработка чанка
20/4286 Обработка чанка
21/4286 Обработка чанка
22/4286 Обработка чанка
23/4286 Обработка чанка
24/4286 Обработка чанка
25/4286 Обработка чанка
26/4286 Обработка чанка
27/4286 Обработка чанка
28/4286 Обработка чанка
29/4286 Обработка чанка
30/4286 Обработка чанка
31/4286 Обработка чанка
32/4286 Обработка чанка
33/4286 Обработка чанка
34/4286 Обработка чанка
35/4286 Обработка чанка
36/4286 Обработка чанка
37/4286 Обработка чанка
38/4286 Обработка чанка
39/4286 Обработка чанка
40/4286 Обработка чанка
41/4286 Обработка чанка
42/4286 Обработка чанка
4

In [10]:
save_to_chroma(fixed_chunks)

C:\Users\kopys\AppData\Local\Temp\ipykernel_12088\3628615760.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings_hf = HuggingFaceEmbeddings(
c:\Users\kopys\.conda\envs\lm-ai\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Saved 4286 chunks to msd_chroma.


C:\Users\kopys\AppData\Local\Temp\ipykernel_12088\2932227189.py:13: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db.persist()
